## Load example PDF file

In [0]:
import urllib.request

url = f"https://raw.githubusercontent.com/StabRise/spark-pdf/refs/heads/main/examples/example1.pdf"
urllib.request.urlretrieve(url, "/dbfs/example1.pdf")
dbutils.fs.cp ("file:/dbfs/example1.pdf", "/FileStore/example1.pdf")


True

In [0]:
dbutils.fs.ls("/FileStore/")

[FileInfo(path='dbfs:/FileStore/example1.pdf', name='example1.pdf', size=115434, modificationTime=1733212447000),
 FileInfo(path='dbfs:/FileStore/jars/', name='jars/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/FileStore/tables/', name='tables/', size=0, modificationTime=0)]

## Try to read file using  `binaryFile` DataSource

In [0]:
df = spark.read.format("binaryFile").load("/FileStore/example1.pdf").select("path").show()

+--------------------+
|                path|
+--------------------+
|dbfs:/FileStore/e...|
+--------------------+



## Try to read file using `PDF` DataSource

In [0]:
spark.read.format("pdf").load("/FileStore/example1.pdf").select("path").show()

---------------------------------------------------------------------------
Py4JJavaError                             Traceback (most recent call last)
File <command-1841387322034103>, line 1
----> 1 spark.read.format("pdf").load("/FileStore/example1.pdf").select("path").show()

File /databricks/spark/python/pyspark/instrumentation_utils.py:47, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     45 start = time.perf_counter()
     46 try:
---> 47     res = func(*args, **kwargs)
     48     logger.log_success(
     49         module_name, class_name, function_name, time.perf_counter() - start, signature
     50     )
     51     return res

File /databricks/spark/python/pyspark/sql/dataframe.py:1061, in DataFrame.show(self, n, truncate, vertical)
    972 def show(self, n: int = 20, truncate: Union[bool, int] = True, vertical: bool = False) -> None:
    973     """
    974     Prints the first ``n`` rows of the DataFrame to the console.
    975 
   (...)
   1059     name | This is a